<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/NN_Dense_2_30_Relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from tabulate import tabulate

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, callbacks, regularizers

from scipy.stats import entropy

# Bayesian Network Data Generation 1000, 2000, ..., 10000 Samples (dense)

In [30]:
# Function to generate CPDs
def generate_cpds():
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random probabilities for EI given IR
    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for SP given IR and EI
    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return ir_probs, ei_given_ir_probs, sp_probs

# Function to generate and save samples
def generate_and_save_samples(ir_probs, ei_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample `IR` state based on `IR` probabilities
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        # Sample `EI` state based on `EI` probabilities given `IR`
        ei_probs_given_ir = ei_probs[:, ir_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs_given_ir[ei_state_idx]

        # Sample `SP` state based on `SP` probabilities given `IR` and `EI`
        sp_probs_given_ir_ei = sp_probs[:, ir_state_idx, ei_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
        sp_prob = sp_probs_given_ir_ei[sp_state_idx]

        # Append sample data to output list
        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_ir_ei]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}'
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes from 1000 to 10000 every 1000
sample_sizes = range(1000, 11000, 1000)

for size in sample_sizes:
    # Generate the CPDs
    ir_probs, ei_given_ir_probs, sp_probs = generate_cpds()

    # Generate and save individual samples for the given sample size
    generate_and_save_samples(ir_probs, ei_given_ir_probs, sp_probs, size, f'combined_probabilities_{size}.csv')

# Notify the user that the process is done
print("\nGeneration and saving of individual samples complete for all sample sizes!")


Sample size: 1000 - First few rows of generated samples:

+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|    | IR_State   |   IR_Prob | EI_State   |   EI_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability |
+====+============+===========+============+===========+=================================================+===================+=========================+
|  0 | low        |    0.7221 | good       |    0.2946 | 0.4488, 0.0269, 0.5243                          | decrease          |                  0.4488 |
+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|  1 | medium     |    0.2256 | average    |    0.0532 | 0.1407, 0.3289, 0.5304                          | decrease          |                  0.1407 |
+----+------------+----

# Hypothesis Model: 1000, 2000, ..., 10000 Samples (dense) 2 hidden Layers, 30 Neurons Relu

In [31]:
# Sample sizes to loop through
sample_sizes = range(1000, 11000, 1000)

# Define the Neural Network architecture with L2 regularization
def create_nn_model(hidden_layers=2, nodes_per_layer=30, l2_lambda=0.01):
    model = models.Sequential()

    # Input layer (2 input features: IR_encoded and EI_encoded)
    model.add(layers.InputLayer(input_shape=(2,)))

    # Hidden layers with L2 regularization and Dropout
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(
            nodes_per_layer,
            activation='relu',
            kernel_regularizer=regularizers.l2(l2_lambda),  # L2 regularization
            name=f"hidden_layer_{layer_num + 1}"
        ))
        model.add(layers.Dropout(0.2))  # Dropout layer to reduce overfitting

    # Output layer (3 classes: decrease, stable, increase) with L2 regularization
    model.add(layers.Dense(
        3,
        activation='softmax',
        kernel_regularizer=regularizers.l2(l2_lambda),  # L2 regularization
        name="output_layer"
    ))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Prepare a dictionary to store the extracted data for each sample size
extracted_data = {}

# Extract the required columns from all sample sizes first
for size in sample_sizes:
    # Load data for the current sample size (adjust the file paths if necessary)
    outcomes_file = f'combined_probabilities_{size}.csv'
    df = pd.read_csv(outcomes_file)

    # Extract only the required columns
    required_columns = ['IR_State', 'EI_State', 'Chosen_SP_State']
    df_extracted = df[required_columns]

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_extracted['IR_encoded'] = df_extracted['IR_State'].map(ir_map)
    df_extracted['EI_encoded'] = df_extracted['EI_State'].map(ei_map)
    df_extracted['SP_encoded'] = df_extracted['Chosen_SP_State'].map(sp_map)

    # Store the extracted and encoded data for later use
    extracted_data[size] = df_extracted

# Loop through each sample size for NN training, validation, and testing
for size in sample_sizes:
    # Retrieve the extracted data for the current sample size
    df = extracted_data[size]

    # Features (IR and EI) and labels (SP)
    X = df[['IR_encoded', 'EI_encoded']]
    y = df['SP_encoded']

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Show split confirmation
    print(f"\nSample size: {size}")
    print("Training Data:", X_train.shape, y_train.shape)
    print("Validation Data:", X_val.shape, y_val.shape)
    print("Test Data:", X_test.shape, y_test.shape)

    # Create the Neural Network model with L2 regularization
    nn_model = create_nn_model(hidden_layers=2, nodes_per_layer=30, l2_lambda=0.01)

    # Early stopping callback to prevent overfitting
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = nn_model.fit(X_train, y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(X_val, y_val),
                           callbacks=[early_stopping],
                           verbose=0)  # Set verbose=0 to avoid too much output

    # Print training, validation, and test accuracy
    train_loss, train_accuracy = nn_model.evaluate(X_train, y_train, verbose=0)
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"Training Accuracy for {size} samples: {train_accuracy:.4f}")
    print(f"Validation Accuracy for {size} samples: {val_accuracy:.4f}")
    print(f"Test Accuracy for {size} samples: {test_accuracy:.4f}")

    # Make predictions on the test set
    predictions = nn_model.predict(X_test)

    # Convert the predicted probabilities to class labels
    predicted_classes = predictions.argmax(axis=1)

    # Create a list to map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']

    # Convert the predicted classes to the original labels
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create a DataFrame for the predicted probabilities
    probs_df = pd.DataFrame(predictions, columns=['Prob_decrease', 'Prob_stable', 'Prob_increase'])

    # Output the IR, EI, predicted SP, and the NN probabilities
    result_df = pd.DataFrame({
        'IR': df['IR_State'].iloc[X_test.index],  # IR column from the original dataframe for the test set
        'EI': df['EI_State'].iloc[X_test.index],  # EI column from the original dataframe for the test set
        'Predicted_SP': predicted_labels           # Predicted SP labels
    })

    # Combine the result with the predicted probabilities
    combined_df = pd.concat([result_df.reset_index(drop=True), probs_df.reset_index(drop=True)], axis=1)

    # Save the test data with predictions to a CSV file
    combined_df.to_csv(f'test_data_nn_{size}.csv', index=False)

    # Show the first few rows of the results for this sample size
    print(f"\nPredicted Results and Probabilities for {size} samples (First 15 rows):")
    print(combined_df.head(15))

# After the loop is done, print this message
print("\nLooping through all sample sizes complete!")


Sample size: 1000
Training Data: (700, 2) (700,)
Validation Data: (150, 2) (150,)
Test Data: (150, 2) (150,)


<ipython-input-31-50d3f9722821>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['IR_encoded'] = df_extracted['IR_State'].map(ir_map)
<ipython-input-31-50d3f9722821>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['EI_encoded'] = df_extracted['EI_State'].map(ei_map)
<ipython-input-31-50d3f9722821>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

Training Accuracy for 1000 samples: 0.4171
Validation Accuracy for 1000 samples: 0.4467
Test Accuracy for 1000 samples: 0.4400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

Predicted Results and Probabilities for 1000 samples (First 15 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium  average       stable       0.371270     0.410251       0.218479
1      low  average     decrease       0.406325     0.403491       0.190184
2      low  average     decrease       0.406325     0.403491       0.190184
3      low     poor       stable       0.392101     0.504868       0.103031
4      low     poor       stable       0.392101     0.504868       0.103031
5      low     good     increase       0.391165     0.204782       0.404053
6   medium     good     decrease       0.366952     0.282036       0.351012
7      low  average     decrease       0.406325     0.403491       0.190184
8      low     good     increase       0.391165     0.204782       0.404053
9   

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training Accuracy for 2000 samples: 0.4836
Validation Accuracy for 2000 samples: 0.4267
Test Accuracy for 2000 samples: 0.4700
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 

Predicted Results and Probabilities for 2000 samples (First 15 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0     high     poor     increase       0.216772     0.256080       0.527148
1   medium     poor     increase       0.284164     0.264311       0.451525
2     high     good     increase       0.235246     0.282821       0.481934
3     high  average     increase       0.230416     0.271109       0.498475
4   medium  average     increase       0.312665     0.272301       0.415034
5   medium  average     increase       0.312665     0.272301       0.415034
6      low     poor     increase       0.332034     0.268666       0.399300
7     high     good     increase       0.235246     0.282821       0.481934
8     high  average     increase       0.230416     0.271109       0.498475
9 

...

Training Accuracy for 8000 samples: 0.3896
Validation Accuracy for 8000 samples: 0.4175
Test Accuracy for 8000 samples: 0.4025
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 8000 samples (First 15 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     good       stable        0.35901     0.392064       0.248926
1   medium  average       stable        0.35901     0.392064       0.248926
2   medium     poor       stable        0.35901     0.392064       0.248926
3     high     poor       stable        0.35901     0.392064       0.248926
4   medium  average       stable        0.35901     0.392064       0.248926
5   medium     good       stable        0.35901     0.392064       0.248926
6   medium     good       stable        0.35901     0.392064       0.248926
7   medium  average       stable        0.35901     0.392064       0.248926
8   medium     good       stable        0.35901     0.392064       0.248926
9  

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training Accuracy for 9000 samples: 0.4790
Validation Accuracy for 9000 samples: 0.4919
Test Accuracy for 9000 samples: 0.4948
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9000 samples (First 15 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low  average     increase       0.149213     0.392157       0.458631
1      low  average     increase       0.149213     0.392157       0.458631
2      low  average     increase       0.149213     0.392157       0.458631
3      low     good     increase       0.431455     0.067790       0.500755
4      low     good     increase       0.431455     0.067790       0.500755
5      low  average     increase       0.149213     0.392157       0.458631
6     high     poor       stable       0.408981     0.421360       0.169659
7   medium     poor       stable       0.249638     0.434385       0.315977
8   medium     poor       stable       0.249638     0.434385       0.315977
9  

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Training Accuracy for 10000 samples: 0.4907
Validation Accuracy for 10000 samples: 0.5027
Test Accuracy for 10000 samples: 0.4820
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 10000 samples (First 15 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low     good       stable       0.256492     0.387760       0.355747
1   medium     good     decrease       0.430681     0.184500       0.384820
2   medium  average     decrease       0.422819     0.180828       0.396353
3      low     good       stable       0.256492     0.387760       0.355747
4     high     poor     decrease       0.528798     0.267069       0.204132
5     high     poor     decrease       0.528798     0.267069       0.204132
6   medium  average     decrease       0.422819     0.180828       0.396353
7   medium  average     decrease       0.422819     0.180828       0.396353
8     high  average     decrease       0.530267     0.268823       0.200909

# K-L Divergence NN Dense Data

In [32]:
# Sample sizes to loop through
sample_sizes = range(1000, 11000, 1000)

# Prepare a list to store K-L divergence results
kl_divergence_results = []

# Loop through each sample size
for size in sample_sizes:
    print(f"\nProcessing sample size: {size}")

    # Load the combined BN data for the current sample size
    combined_data_bn = pd.read_csv(f'combined_probabilities_{size}.csv')

    # Split the data into train, validation, and test sets
    X = combined_data_bn[['IR_State', 'EI_State']]
    y = combined_data_bn[['Chosen_SP_State', 'SP_Probabilities (decrease, stable, increase)']]

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Get the test indices
    test_indices = X_test.index

    # Get the corresponding rows from the combined BN data using the test indices
    bn_test_data = combined_data_bn.loc[test_indices]

    # Load the corresponding NN test data for the current sample size
    nn_test_data = pd.read_csv(f'test_data_nn_{size}.csv')

    # Extract NN predicted probabilities and BN ground truth probabilities
    nn_probs = nn_test_data[['Prob_decrease', 'Prob_stable', 'Prob_increase']].values
    bn_probs = bn_test_data['SP_Probabilities (decrease, stable, increase)'].apply(
        lambda x: np.array(list(map(float, x.strip('[]').split(','))))
    ).values

    # Calculate K-L divergence between NN predicted probabilities and BN ground truth probabilities
    kl_divergences = []
    output_data = []  # For tabulating output

    for i in range(len(nn_probs)):
        nn_prob = nn_probs[i]
        bn_prob = bn_probs[i]

        # Ensure both are valid probability distributions
        epsilon = 1e-10
        nn_prob = np.clip(nn_prob, epsilon, 1)
        bn_prob = np.clip(bn_prob, epsilon, 1)

        # Normalize to ensure they sum to 1
        nn_prob /= nn_prob.sum()
        bn_prob /= bn_prob.sum()

        # Compute K-L divergence
        kl_div = entropy(bn_prob, nn_prob)
        kl_divergences.append(kl_div)

        # Add data to output for tabulation
        output_data.append({
            'Sample_Index': i,
            'IR': bn_test_data.iloc[i]['IR_State'],
            'EI': bn_test_data.iloc[i]['EI_State'],
            'Ground_Truth_Probs': ', '.join([f'{prob:.4f}' for prob in bn_prob]),
            'NN_Probs': ', '.join([f'{prob:.4f}' for prob in nn_prob]),
            'KL_Divergence': f'{kl_div:.4f}'
        })

    # Create a DataFrame for the output data and tabulate the first few rows
    output_df = pd.DataFrame(output_data)
    print(f"\nK-L Divergence Results for {size} samples (First 5 rows):\n")
    print(tabulate(output_df.head(5), headers='keys', tablefmt='grid'))

    # Calculate and display the average K-L divergence for this sample size
    average_kl_divergence = np.mean(kl_divergences)
    std_kl_divergence = np.std(kl_divergences)
    print(f"\nAverage K-L Divergence for {size} samples: {average_kl_divergence:.4f}, Std Dev: {std_kl_divergence:.4f}")

    # Append the results to the list
    kl_divergence_results.append({
        'Sample_Size': size,
        'Average_KL_Divergence': average_kl_divergence,
        'Std_Dev': std_kl_divergence
    })

# Save the K-L divergence results to a CSV file
kl_divergence_df = pd.DataFrame(kl_divergence_results)
kl_divergence_df.to_csv('kl_div_NN_2_30_dense.csv', index=False)

print("\nAll sample sizes have been processed and K-L divergences calculated. Results saved to 'kl_div_NN_2_30_dense.csv'.")


Processing sample size: 1000

K-L Divergence Results for 1000 samples (First 5 rows):

+----+----------------+--------+---------+------------------------+------------------------+-----------------+
|    |   Sample_Index | IR     | EI      | Ground_Truth_Probs     | NN_Probs               |   KL_Divergence |
+====+================+========+=========+========================+========================+=================+
|  0 |              0 | medium | average | 0.1407, 0.3289, 0.5304 | 0.3713, 0.4103, 0.2185 |          0.2612 |
+----+----------------+--------+---------+------------------------+------------------------+-----------------+
|  1 |              1 | low    | average | 0.3653, 0.5140, 0.1207 | 0.4063, 0.4035, 0.1902 |          0.0306 |
+----+----------------+--------+---------+------------------------+------------------------+-----------------+
|  2 |              2 | low    | average | 0.3653, 0.5140, 0.1207 | 0.4063, 0.4035, 0.1902 |          0.0306 |
+----+----------------+-